In [24]:
import json

import geopandas as gpd
import networkx as nx
import numpy as np
import pandas as pd
import shapely
import structlog
import yaml
from shapely import Point, Polygon


def compute_ellipticity(points: np.array) -> float:
    """
    Compute ellipticity of a set of points.

    Parameters:
    - points (numpy array): Array of shape (n, 2) representing (x, y) coordinates of points.

    Returns:
    - ellipticity (float): Ellipticity value.
    """

    # Calculate the covariance matrix of the points
    cov_matrix = np.cov(points, rowvar=False)

    # Calculate eigenvalues and eigenvectors of the covariance matrix
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

    # Sort eigenvalues in descending order
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors = eigenvectors[:, sorted_indices]

    # Major and minor axis lengths are square roots of eigenvalues
    major_axis_length = np.sqrt(eigenvalues[0])
    minor_axis_length = np.sqrt(eigenvalues[1])

    # Compute ellipticity
    ellipticity = 1.0 - (minor_axis_length / major_axis_length)

    return ellipticity


def ellipticity(points: list[Point], threshold: int = 10, decimals: int = 4) -> float:
    points = [(i.x, i.y) for i in points]
    if len(points) < threshold:
        return None

    return np.round(compute_ellipticity(points), decimals)


def determine_stop_geometries(
    stops: gpd.GeoDataFrame,
    subgraphs: dict,
    time_marker: int = 39,
    suffix: str = "",
    concaveness_ratio: float = 0.2,
    include_empty: bool = False,
) -> pd.DataFrame:
    """
    Calculates convex and concave hulls of the accessible network, and also the ellipticity of the stops.

    While the convex hull is unambiguous, multiple concave hulls can be constructed.
    """
    records = []
    for row in stops.itertuples():
        accessible_stops = list(
            subgraphs.get(f"{row.stop_id}_network_{time_marker}", nx.Graph())
        )
        accessible_stops = stops[stops["stop_id"].isin(accessible_stops)].copy()
        if len(accessible_stops) == 0:
            if include_empty:
                records.append([row.stop_id, Polygon(), 0, Polygon(), 0, 0])
            continue
        points = accessible_stops.union_all()
        cv = shapely.convex_hull(points)
        cc = shapely.concave_hull(points, ratio=concaveness_ratio)
        el = ellipticity(accessible_stops.geometry.tolist())

        records.append(
            [
                row.stop_id,
                cv,
                round(cv.area / 1e6, 3),
                cc,
                round(cc.area / 1e6, 3),
                el,
            ]
        )
    columns = ["stop_id"] + [
        i + suffix
        for i in [
            "convex",
            "convex_area",
            "concave",
            "concave_area",
            "ellipticity",
        ]
    ]

    return pd.DataFrame.from_records(records, columns=columns)


def determine_stop_geometries_from_walk(
    stops: gpd.GeoDataFrame,
    isochrones: gpd.GeoDataFrame,
    accessible_stops,
    crs: int = 23700,
    ellipticity_threshold: int = 2,
) -> gpd.GeoDataFrame:
    records = []
    for row in stops.itertuples():
        if row.stop_id not in accessible_stops:
            # logger.info(row.stop_id)
            continue
        accessible = stops[stops["stop_id"].isin(accessible_stops[row.stop_id])].copy()

        el = ellipticity(accessible.geometry.tolist(), threshold=ellipticity_threshold)
        accessible_area = isochrones[
            (isochrones["stop_id"].isin(accessible_stops[row.stop_id]))
            & (isochrones["costing"] == "walk")
            & (isochrones["range"] == 5)
        ].copy()
        accessible_area_crs = accessible_area.to_crs(crs).union_all()
        records.append(
            [
                row.stop_id,
                accessible_area.union_all(),
                round(accessible_area_crs.area / 1e6, 3),
                el,
                len(accessible),
            ]
        )
    df = pd.DataFrame.from_records(
        records,
        columns=[
            "stop_id",
            "geometry",
            "area",
            "ellipticity",
            "number_of_accessible_stops",
        ],
    )
    return gpd.GeoDataFrame(df, crs=4326)

In [25]:
with open("../data/crs.yaml", "r") as fp:
    crs = yaml.safe_load(fp)

logger = structlog.get_logger()

In [26]:
CITY = "helsinki"
ELLIPTICITY_THRESHOLD = 5

In [27]:
with open(f"../data/stops/{CITY}/accessible_stops.json", "r") as fp:
    accessible_stops = json.load(fp)

if CITY in ["paris"]:
    accessible_stops = {int(float(k)): v for k, v in accessible_stops.items()}

In [28]:
all_stops = set([i for k, v in accessible_stops.items() for i in v])
len(all_stops)

6052

In [29]:
isochrones = pd.read_csv(f"../output/{CITY}/isochrones.csv", dtype={"stop_id": str})
isochrones["geometry"] = isochrones["geometry"].apply(shapely.from_wkt)
isochrones = gpd.GeoDataFrame(isochrones, geometry="geometry", crs=4326)

In [30]:
stops = pd.read_csv(
    f"../data/stops/{CITY}/stops_with_centrality.csv",
    engine="pyarrow",
    dtype={"stop_id": str},
)
stops["geometry"] = stops.apply(lambda x: Point(x["stop_lon"], x["stop_lat"]), axis=1)
stops = gpd.GeoDataFrame(stops, geometry="geometry", crs=4326)
stops.head(2)

,Node,Eigenvector Centrality,Degree Centrality,Closeness Centrality,Betweenness Centrality,stop_id,clust,stop_lat,stop_lon,stop_name,geometry
0,2.0,2.163418e-09,0.000278,0.043629,0.0,5020503,2,60.976337,25.657594,Lahti,POINT (25.65759 60.97634)
1,2.0,2.163418e-09,0.000278,0.043629,0.0,5020553,2,60.976337,25.657575,Lahti,POINT (25.65758 60.97634)


In [31]:
isochrones.query("costing == 'walk' & range == 5")

,stop_id,geometry,costing,range
0,5020503,"POLYGON ((25.65959 60.97988, 25.65578 60.97915...",walk,5
6,5020553,"POLYGON ((25.65958 60.97988, 25.65658 60.97937...",walk,5
12,5020502,"POLYGON ((25.3108 60.65116, 25.30945 60.65079,...",walk,5
18,5020552,"POLYGON ((25.3108 60.65116, 25.30945 60.65079,...",walk,5
24,5010505,"POLYGON ((24.85775 60.6335, 24.85475 60.63399,...",walk,5
...,...,...,...,...
44190,9212216,"POLYGON ((25.4921 60.25615, 25.49048 60.25475,...",walk,5
44196,9300201,"POLYGON ((25.66387 60.39656, 25.66287 60.39692...",walk,5
44202,9300204,"POLYGON ((25.66705 60.39642, 25.66605 60.39622...",walk,5
44208,9400206,"POLYGON ((25.53693 60.49888, 25.53547 60.49894...",walk,5


In [32]:
sgfw = determine_stop_geometries_from_walk(
    stops,
    isochrones.query("costing == 'walk' & range == 5"),
    accessible_stops,
    crs=crs[CITY],
    ellipticity_threshold=ELLIPTICITY_THRESHOLD,
)
sgfw.to_csv(f"../output/{CITY}/stop_geometries_from_walk.csv", index=False)
sgfw.to_file(f"../output/{CITY}/stop_geometries_from_walk.geojson")

In [33]:
sgfw

,stop_id,geometry,area,ellipticity,number_of_accessible_stops
0,9300202,"POLYGON ((25.48089 60.40198, 25.47589 60.40224...",0.195,NaN,1
1,9300203,"POLYGON ((25.48241 60.40294, 25.48141 60.40293...",0.186,NaN,1
2,9221209,"POLYGON ((25.35389 60.37758, 25.3532 60.37703,...",0.219,NaN,1
3,9228211,"MULTIPOLYGON (((25.30128 60.36845, 25.30243 60...",2.068,0.8886,11
4,5010201,"MULTIPOLYGON (((25.03363 60.4043, 25.03485 60....",1.002,NaN,3
...,...,...,...,...,...
6047,6070232,"MULTIPOLYGON (((24.34739 60.09798, 24.34796 60...",1.883,0.8062,11
6048,6100206,"POLYGON ((24.36454 60.02551, 24.36267 60.02681...",1.769,0.7651,12
6049,1492103,"POLYGON ((25.05276 60.16043, 25.05292 60.16001...",2.329,0.8299,12
6050,9300201,"POLYGON ((25.66387 60.39656, 25.66287 60.39692...",0.408,NaN,1


In [34]:
# stop = "009461"
# # stop = "009749"
# fig, ax = plt.subplots()
# sgfw[sgfw["stop_id"] == stop].plot(ax=ax, fc="#afdfff", ec="#00aaff")
# stops[
#     (stops["stop_id"].isin(accessible_stops[stop])) & (stops["stop_id"] != stop)
# ].plot(ax=ax, color="#2d2d2d", markersize=15, zorder=5)
# stops[stops["stop_id"] == stop].plot(ax=ax, color="red", markersize=20, zorder=10)